## Using bert-kor-base and fine tune

In [4]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
import pandas as pd

review_label = pd.read_csv("finetune_label.csv")


tokenizer = BertTokenizer.from_pretrained('kykim/bert-kor-base')

# 책 리뷰 데이터셋 클래스 정의
class BookReviewDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = int(self.labels[idx])

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

review_label = pd.read_csv("finetune_label.csv")

review_texts = review_label['Review_Text'].tolist()
review_labels = review_label['label'].tolist()

# Review 데이터셋 초기화
review_dataset = BookReviewDataset(review_texts, review_labels, tokenizer)

# BERT 모델 및 감정 분석을 위한 파인튜닝
model = BertForSequenceClassification.from_pretrained('kykim/bert-kor-base', num_labels=2)  # 2개의 감정 클래스
optimizer = AdamW(model.parameters(), lr=1e-5)

# DataLoader 설정
book_review_loader = DataLoader(review_dataset, batch_size=2, shuffle=True)  # 변수명 수정

# 파인튜닝 실행
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in book_review_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# 파인튜닝이 완료된 모델을 저장
model.save_pretrained('book_finetuning_model2')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at kykim/bert-kor-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\heo kang\anaconda3\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### Using original model  with 200reviews

In [5]:
import pandas as pd

# CSV 파일 읽기
file_path = "all_review.csv"
reviews_df = pd.read_csv(file_path)

# 리뷰 텍스트 데이터 추출
reviews_texts = reviews_df["Review_Text"].tolist()

# 사전 훈련된 BERT 모델과 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained('kykim/bert-kor-base')
model = BertForSequenceClassification.from_pretrained('kykim/bert-kor-base')

# 추론을 위한 함수 정의
def predict_sentiment(review_text):
    inputs = tokenizer(review_text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()
    return predicted_class, probabilities[0][predicted_class].item()

# 리뷰에 대한 감정 분석 결과 출력
for i, review_text in enumerate(reviews_texts[-200:], start=1):
    predicted_class, confidence = predict_sentiment(review_text)
    sentiment = "긍정" if predicted_class == 1 else "부정"
    print(f"Review {i}: {review_text}")
    print(f"감정: {sentiment}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at kykim/bert-kor-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Review 1: 읽다가 주변인들이 너무 무능해서 짜증이나
감정: 긍정
Review 2: 재미있네요. 화골여오작에 비해 왕을 도와 정적을 제거하는 정치적 이야기의 비중이 더 큼
감정: 긍정
Review 3: 여주혼자 사건 다 해결. 주위인물들은 왜이리 무능한지
감정: 긍정
Review 4: 담담하지만 넓게 펼쳐지는 스토리~ 잼있어요
감정: 부정
Review 5: 재미있습니다 ~
감정: 긍정
Review 6: 복수의 시작인가요~
감정: 부정
Review 7: 눈치없는 페하와 지혜로운 법의의 사랑은 진전이 없네용~ㅎ
감정: 긍정
Review 8: 개기는 놈의 끝은? ㅎ
감정: 긍정
Review 9: 페하의 신랑각시놀이~
감정: 부정
Review 10: 드러나는 진실~
감정: 부정
Review 11: 긴 여정이였는데 마무리가 조금 아쉽네요
그래도 잼있게 잘 보았습니다 ~
감정: 긍정
Review 12: 로맨스가 많이 없어서 좀 아쉽네요
감정: 부정
Review 13: 재미있게 잘 읽었습니다. 고맙습니다 ^^
감정: 부정
Review 14: 재구독방지용
감정: 부정
Review 15: 작가가 쓰기 싫은지 후반을 던졌음
감정: 부정
Review 16: 아주 재밌게봤습니다.
감정: 부정
Review 17: 중국작가의 책이라길래 별 기대가 없었는데 재밌게 잘 봤습니다.
감정: 긍정
Review 18: 🍎🍠🍠  초반은 신선했는데, 온통 귀신이야기네. 번잡하여 중도에 하차.
감정: 긍정
Review 19: 나도 도전
감정: 부정
Review 20: 완독. 재미있게 잘 봤습니다.
감정: 긍정
Review 21: 길지 않게 느껴지고 볼만합니다
감정: 부정
Review 22: 중국이라
선입견이있었는데
볼만하네요
감정: 부정
Review 23: 재미있게 잘 읽었습니다
감정: 부정
Review 24: 독특한 소재와 계속되는 사건으로 시간가는줄 모르고 읽었네요.너무 재미있었요!!
감정: 긍정
Review 25: 신선한 소재에다가 필력 좋음 대신 초반에 몰입이 어렵긴함
감정: 긍정
Rev

Review 198: 프롤로그는 임팩트가 있고 호기심을 자극해야 읽게 되는데  읽고싶은 요소가 없음. 읽덮
감정: 긍정
Review 199: 재밌어요~
감정: 긍정
Review 200: 재밌는데.. 아니 여주언니죽었다고하지않았나요?책여러군데에죽었다고나오든데..뭐지ㅡㅡ;;
감정: 긍정


### Using new model with 200 reviews

In [6]:
import pandas as pd

# CSV 파일 읽기
file_path = "all_review.csv"
reviews_df = pd.read_csv(file_path)

# 리뷰 텍스트 데이터 추출
reviews_texts = reviews_df["Review_Text"].tolist()

# 사전 훈련된 BERT 모델과 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained('kykim/bert-kor-base')
model = BertForSequenceClassification.from_pretrained('book_finetuning_model2')

# 추론을 위한 함수 정의
def predict_sentiment(review_text):
    inputs = tokenizer(review_text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()
    return predicted_class, probabilities[0][predicted_class].item()

# 리뷰에 대한 감정 분석 결과 출력
for i, review_text in enumerate(reviews_texts[-200:], start=1):
    predicted_class, confidence = predict_sentiment(review_text)
    sentiment = "긍정" if predicted_class == 1 else "부정"
    print(f"Review {i}: {review_text}")
    print(f"감정: {sentiment}")

Review 1: 읽다가 주변인들이 너무 무능해서 짜증이나
감정: 부정
Review 2: 재미있네요. 화골여오작에 비해 왕을 도와 정적을 제거하는 정치적 이야기의 비중이 더 큼
감정: 긍정
Review 3: 여주혼자 사건 다 해결. 주위인물들은 왜이리 무능한지
감정: 부정
Review 4: 담담하지만 넓게 펼쳐지는 스토리~ 잼있어요
감정: 긍정
Review 5: 재미있습니다 ~
감정: 긍정
Review 6: 복수의 시작인가요~
감정: 긍정
Review 7: 눈치없는 페하와 지혜로운 법의의 사랑은 진전이 없네용~ㅎ
감정: 부정
Review 8: 개기는 놈의 끝은? ㅎ
감정: 부정
Review 9: 페하의 신랑각시놀이~
감정: 긍정
Review 10: 드러나는 진실~
감정: 긍정
Review 11: 긴 여정이였는데 마무리가 조금 아쉽네요
그래도 잼있게 잘 보았습니다 ~
감정: 긍정
Review 12: 로맨스가 많이 없어서 좀 아쉽네요
감정: 부정
Review 13: 재미있게 잘 읽었습니다. 고맙습니다 ^^
감정: 긍정
Review 14: 재구독방지용
감정: 긍정
Review 15: 작가가 쓰기 싫은지 후반을 던졌음
감정: 부정
Review 16: 아주 재밌게봤습니다.
감정: 긍정
Review 17: 중국작가의 책이라길래 별 기대가 없었는데 재밌게 잘 봤습니다.
감정: 긍정
Review 18: 🍎🍠🍠  초반은 신선했는데, 온통 귀신이야기네. 번잡하여 중도에 하차.
감정: 부정
Review 19: 나도 도전
감정: 긍정
Review 20: 완독. 재미있게 잘 봤습니다.
감정: 긍정
Review 21: 길지 않게 느껴지고 볼만합니다
감정: 긍정
Review 22: 중국이라
선입견이있었는데
볼만하네요
감정: 긍정
Review 23: 재미있게 잘 읽었습니다
감정: 긍정
Review 24: 독특한 소재와 계속되는 사건으로 시간가는줄 모르고 읽었네요.너무 재미있었요!!
감정: 긍정
Review 25: 신선한 소재에다가 필력 좋음 대신 초반에 몰입이 어렵긴함
감정: 부정
Rev

Review 197: 그리고남주가못된넘인건맞는데,언니일은 남주탓은아니지않나? 근데 왜 언니일을두고계속남주탓하지
감정: 부정
Review 198: 프롤로그는 임팩트가 있고 호기심을 자극해야 읽게 되는데  읽고싶은 요소가 없음. 읽덮
감정: 부정
Review 199: 재밌어요~
감정: 긍정
Review 200: 재밌는데.. 아니 여주언니죽었다고하지않았나요?책여러군데에죽었다고나오든데..뭐지ㅡㅡ;;
감정: 부정


### Apply to real review data

In [ ]:
# CSV 파일 읽기
file_path = "all_review.csv"
reviews_df = pd.read_csv(file_path)

# 리뷰 텍스트 데이터 추출
reviews_texts = reviews_df["Review_Text"].astype(str).tolist()  # 데이터 타입을 문자열로 변환

# 사전 훈련된 BERT 모델과 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained('kykim/bert-kor-base')
model = BertForSequenceClassification.from_pretrained('book_finetuning_model2')

# 추론을 위한 함수 정의
def predict_sentiment(review_text):
    inputs = tokenizer(review_text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()
    return predicted_class, probabilities[0][predicted_class].item()

# 리뷰에 대한 감정 분석 결과 저장
results = []

for i, review_text in enumerate(reviews_texts, start=1):
    predicted_class, confidence = predict_sentiment(review_text)
    sentiment = "긍정" if predicted_class == 1 else "부정"
    results.append({"Review": review_text, "감정": sentiment})

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)

# CSV 파일로 저장
output_path = "all_review_sentiment_results.csv"
results_df.to_csv(output_path, index=False)

print(f"결과가 {output_path}에 저장되었습니다.")

### transfer positive -> 1 , negative -> 0 

In [ ]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv('all_review_labeled.csv')

# 'sentiment' 열의 값을 1로 변환 (긍정인 경우)
df['Column2.1'] = df['Column2.1'].apply(lambda x: 1 if x == '긍정' else 0)

# 새로운 CSV 파일로 저장
output_csv_path = 'all_review_labeled_bi.csv'
df.to_csv(output_csv_path, index=False)

print(f'Data has been saved to {output_csv_path}')